# Operating on Data in Pandas

One of the essential pieces of NumPy is the ability to perform quick element-wise operations, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.).
Pandas inherits much of this functionality from NumPy, and the ufuncs that we introduced in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) are key to this.

Pandas includes a couple useful twists, however: for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output, and for binary operations such as addition and multiplication, Pandas will automatically *align indices* when passing the objects to the ufunc.
This means that keeping the context of data and combining data from different sources–both potentially error-prone tasks with raw NumPy arrays–become essentially foolproof ones with Pandas.
We will additionally see that there are well-defined operations between one-dimensional ``Series`` structures and two-dimensional ``DataFrame`` structures.


Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas a nivel de elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.). Pandas hereda gran parte de esta funcionalidad de NumPy, y los ufuncs que presentamos en Computation on NumPy Arrays: Universal Functions son clave para esto.

Sin embargo, Pandas incluye un par de giros útiles: para operaciones unarias como negación y funciones trigonométricas, estos ufuncs conservarán las etiquetas de índice y columna en la salida, y para operaciones binarias como la suma y la multiplicación, Pandas alineará automáticamente los índices al pasar los objetos a el ufunc. Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes, ambas tareas potencialmente propensas a errores con matrices NumPy sin procesar, se convierten esencialmente en infalibles con Pandas. Además, veremos que hay operaciones bien definidas entre las estructuras de serie unidimensionales y las estructuras de marcos de datos bidimensionales.

## Ufuncs: Index Preservation

Because Pandas is designed to work with NumPy, any NumPy ufunc will work on Pandas ``Series`` and ``DataFrame`` objects.
Let's start by defining a simple ``Series`` and ``DataFrame`` on which to demonstrate this:


Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc NumPy funcionará en objetos Pandas Series y DataFrame. Comencemos definiendo una serie simple y un marco de datos para demostrar esto:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [25]:
If we apply a NumPy ufunc on either of these objects, the result will be another Pandas object *with the indices preserved:*

Si aplicamos un ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas con los índices preservados:

SyntaxError: invalid syntax (<ipython-input-25-4c1cfaf36da1>, line 1)

In [5]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

Or, for a slightly more complex calculation:


O, para un cálculo un poco más complejo


In [6]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Any of the ufuncs discussed in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) can be used in a similar manner.


Cualquiera de los ufuncs discutidos en Computación en matrices NumPy: Universal Functions se puede usar de manera similar

## UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will align indices in the process of performing the operation.
This is very convenient when working with incomplete data, as we'll see in some of the examples that follow.


UFuncs: Alineación de índice
Para operaciones binarias en dos objetos Series o DataFrame, Pandas alineará índices en el proceso de realizar la operación. Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen

### Index alignment in Series

As an example, suppose we are combining two different data sources, and find only the top three US states by *area* and the top three US states by *population*:


Alineación de índice en serie
Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes y encontramos solo los tres principales estados de EE. UU. Por área y los tres principales estados de EE. UU. Por población:

In [7]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

Let's see what happens when we divide these to compute the population density:


Veamos qué sucede cuando los dividimos para calcular la densidad de población:

In [8]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the *union* of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

La matriz resultante contiene la unión de índices de las dos matrices de entrada, que podrían determinarse utilizando la aritmética de conjunto de Python estándar en estos índices:

In [9]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Any item for which one or the other does not have an entry is marked with ``NaN``, or "Not a Number," which is how Pandas marks missing data (see further discussion of missing data in [Handling Missing Data](03.04-Missing-Values.ipynb)).
This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

Cualquier elemento para el que uno u otro no tenga una entrada está marcado con NaN, o "No es un número", que es cómo Pandas marca los datos faltantes (ver más detalles sobre los datos faltantes en Manejo de datos faltantes). Esta coincidencia de índice se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores faltantes se completan con NaN de forma predeterminada:

In [10]:
A = pd.Series([2, 4, 6], index=[0, 1, 2]) 
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [11]:
# en A la posicion 0 es dos y en B no hay posición 0, con A.add(B,fill_value=0) hago que B en la pos 0 valga 0 y en la pos 3 de B no hay pos 3 en A luego toma el valor de B en esa posición que es 5

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators.
For example, calling ``A.add(B)`` is equivalent to calling ``A + B``, but allows optional explicit specification of the fill value for any elements in ``A`` or ``B`` that might be missing:

Si el uso de valores NaN no es el comportamiento deseado, el valor de relleno se puede modificar utilizando métodos de objeto apropiados en lugar de los operadores. Por ejemplo, llamar a A.add (B) es equivalente a llamar a A + B, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en A o B que pueda faltar:

In [12]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Index alignment in DataFrame

A similar type of alignment takes place for *both* columns and indices when performing operations on ``DataFrame``s:

Alineación de índice en DataFrame
Un tipo similar de alineación tiene lugar tanto para columnas como para índices cuando se realizan operaciones en DataFrames:

In [13]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), #genera nºs aleatorios entre 0 y 20  y losdispone en matriz 2x2
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [14]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), #genera nºs aleatorios entre 0 y 10  y losdispone en matriz 3x3
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [15]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


Notice that indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted.
As was the case with ``Series``, we can use the associated object's arithmetic method and pass any desired ``fill_value`` to be used in place of missing entries.
Here we'll fill with the mean of all values in ``A`` (computed by first stacking the rows of ``A``):

Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos, y los índices en el resultado están ordenados. Como fue el caso con Series, podemos usar el método aritmético del objeto asociado y pasar cualquier valor de relleno deseado que se utilizará en lugar de entradas faltantes. Aquí completaremos con la media de todos los valores en A (calculados apilando primero las filas de A):

In [16]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


The following table lists Python operators and their equivalent Pandas object methods:

La siguiente tabla enumera los operadores de Python y sus métodos de objeto Pandas equivalentes:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operations Between DataFrame and Series

When performing operations between a ``DataFrame`` and a ``Series``, the index and column alignment is similarly maintained.
Operations between a ``DataFrame`` and a ``Series`` are similar to operations between a two-dimensional and one-dimensional NumPy array.
Consider one common operation, where we find the difference of a two-dimensional array and one of its rows:


Ufuncs: operaciones entre DataFrame y Series
Al realizar operaciones entre un DataFrame y una Serie, el índice y la alineación de la columna se mantienen de manera similar. Las operaciones entre un DataFrame y una Serie son similares a las operaciones entre una matriz NumPy bidimensional y unidimensional. Considere una operación común, donde encontramos la diferencia de una matriz bidimensional y una de sus filas:

In [17]:
A = rng.randint(10, size=(3, 4)) #Genera nºs aleatorios rango 10 y los dispone en matriz 3x4
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [18]:
A - A[0] #Resta entre matriz bidinmesional y una de sus filas

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

According to NumPy's broadcasting rules (see [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)), subtraction between a two-dimensional array and one of its rows is applied row-wise.

In Pandas, the convention similarly operates row-wise by default:

De acuerdo con las reglas de transmisión de NumPy (ver Computación en matrices: transmisión), la resta entre una matriz bidimensional y una de sus filas se aplica en forma de fila.

En Pandas, la convención funciona de manera similar en fila por defecto:

In [19]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the ``axis`` keyword:

Si, en cambio, desea operar en forma de columna, puede usar los métodos de objeto mencionados anteriormente, mientras especifica la palabra clave del eje

In [21]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Note that these ``DataFrame``/``Series`` operations, like the operations discussed above, will automatically align  indices between the two elements:
Tenga en cuenta que estas operaciones DataFrame / Series, como las operaciones discutidas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [22]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [23]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


This preservation and alignment of indices and columns means that operations on data in Pandas will always maintain the data context, which prevents the types of silly errors that might come up when working with heterogeneous and/or misaligned data in raw NumPy arrays.


Esta preservación y alineación de índices y columnas significa que las operaciones con datos en Pandas siempre mantendrán el contexto de datos, lo que evita los tipos de errores tontos que pueden surgir al trabajar con datos heterogéneos y / o desalineados en matrices NumPy sin procesar.